In [119]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## **General Strategy for the Project**:

#### Initialization
0) Define and initialize necessary global variables

#### Part #1: Data Filtering and Cleaning
1) Filter out all valid US and CAD Stocks from the provided CSV file (Storing US tickers in another list for currency conversion later on)
2) Download and store closing price, options and volume data in a dictionary using yfinance.
3) Filter out tickers within date range based on given minimum monthly average volume.

#### Part #2: Portfolio Construction
4) Rank stocks based on Standard Deviation of percentage change in returns in descending order.
5) Rank stocks based on PCR values using options data in descending order.
6) Score the stocks based on the two ranks, and create a new ranking based on the scoring.
7) We select stocks based on the ranking and calculate weights that would maximize the portfolio sharpe ratio while beta is within pre-defined constraints.
8) Run sharpe ratio calculation function on portfolio from size 12(min) to 24(max), so that the final portfolio (with weightings) is picked based on highest output sharpe ratio.

#### Part #3: Evaluation and Proof
9) Given the chosen portfolio of xx stocks, graph the change in portfolio standard deviation as other stocks are added.
10) Beta of portfolio compared to the S&P 500.
11) Calculate the Beta between our portfolio and an equally weighted portfolio including all valid stocks.
12) Beta between our portfolio with varied weight versus when the portfolio is equally weighted.
13) Sharpe ratio between our portfolio with varied weight versus when it is equally weighted.
14) Graphically compare sharpe ratios amongst our portfolio of xx stocks and the portfolios of varying 12-24 stocks.

#### Part #4: Final Output
15) Creating the final portfolio dataframe and CSV.

## Initializing Variables

In [120]:
def get_tickers():
    tickers = pd.read_csv('Tickers.csv')
    ticker_lst = [tickers.columns[0]] + (list(tickers[tickers.columns[0]]))
    return ticker_lst

In [121]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Define constants
min_avg_volume = 100000
min_trading_days = 18
start_date, end_date = '2022-09-30', '2024-09-30'
min_stocks, max_stocks = 12, 24

# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = get_tickers()

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)
exchange_rate = yf.Ticker('CAD=X').fast_info['last_price']
print(f'The current exchange rate for the latest available day:\nUSD -> CAD: ${np.round(exchange_rate, 4)}')

The current exchange rate for the latest available day:
USD -> CAD: $1.3989


#### We must filter the tickers csv as follows:
- Must be listed on yfinance
- The currency is listed as USD or CAD 
- 100,000+ average monthly volume trades
- More than 18 trades per month
- Sufficient data

In [122]:
# Drop tickers with less than 18 days of trading data
def drop_short_trading_months(df):
    """
    Drops months with less than 18 trading days from a yfinance history DataFrame.

    Parameters:
        df (pd.DataFrame): A yfinance DataFrame with a DatetimeIndex and stock data.

    Returns:
        pd.DataFrame: Filtered DataFrame with only months having >= 18 trading days.
    """
    # Ensure the index is a DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index must be a DatetimeIndex.")
    
    df = df.copy()  # Avoid modifying the original DataFrame
    df.index = df.index.tz_localize(None)
    # Group by year and month
    df['YearMonth'] = df.index.to_period('M')  # Creates a 'YearMonth' period
    
    # Count trading days for each month
    trading_days_per_month = df.groupby('YearMonth').size()
    
    # Get valid months with at least 18 trading days
    valid_months = trading_days_per_month[trading_days_per_month >= 18].index

    # Filter DataFrame to include only rows in valid months
    filtered_df = df[df['YearMonth'].isin(valid_months)].drop(columns=['YearMonth'])

    return filtered_df

In [123]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    valid_tickers, invalid_tickers = {}, []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        try:
            info = stock.fast_info # Get basic stock info

            hist = stock.history(start=start_date, end=end_date) # Get stock history
            pd.to_datetime(hist.index, format='%Y-%m-%d')

            avg_volume = hist.loc[((hist.index >= start_date) & (hist.index <= end_date))]['Volume'].mean() # Calculate average volume in specified date range.
            currency = info.get("currency")
            
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                ( currency == "USD" or currency == "CAD") and # filter for stocks that are not USD
                (avg_volume >= min_avg_volume)): # Filter by volume greater than 100,000
                if currency == "CAD":
                    hist = drop_short_trading_months(hist) # Drop months with less than 18 trading days
                    hist.index = hist.index.strftime('%Y-%m-%d')
                    valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
                elif currency == "USD":
                    hist = drop_short_trading_months(hist) # Drop months with less than 18 trading days
                    hist.index = hist.index.strftime('%Y-%m-%d')
                    valid_tickers[ticker] = hist['Close'] * exchange_rate # Convert USD to CAD
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers]
    # valid_tickers is a dictionary of Series where the key is the name of the ticker. 
    # invalid_tickers is a list of ticker strings which were removed in the filtering process. 

In [12]:
# sharpe ratio optimization

def optimal_sharpe(tickers, start_date, end_date, risk_free_rate, investment):
    # download data
    data = yf.download(tickers, start=start_date, end=end_date)['Close']

    # calculate mean return of stocks and covariance of stocks
    returns = data.pct_change()
    returns.drop(index=returns.index[0], inplace = True)
    mean_returns = returns.mean()
    covariance_matrix = returns.cov()

    def neg_sharpe(weights):
        #alternate
        #portfolio = data/data.iloc[0] # normalize returns
        #portfolio = portfolio*weights*investment
        #portfolio['total'] = portfolio.sum(axis=1)
        #portfolio['daily return'] = portfolio['total'].pct_change(1)

        #er = portfolio['daily return'].mean()
        #std = portfolio['daily return'].std()
        #sr = er/std
        


        # calculate portfolio expected return by weighing each stock's expected return
        num_days = len(returns)
        portfolio_expected_return = np.sum(weights*mean_returns*num_days)

        portfolio_variance = 0
        # calculate portfolio risk (std) by finding the portfolio variance, which is affected by covariance
        for i in range(len(weights)):
            for j in range(len(weights)):
                portfolio_variance += weights[i] * weights[j] * covariance_matrix.iloc[i, j]*num_days
        portfolio_std = np.sqrt(portfolio_variance)
        
        # calculate sharpe ratio
        sharpe = (portfolio_expected_return - risk_free_rate)/portfolio_std
        
        return -sharpe #make sharpe ratio negative for minimize function

    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints)

    return results


tickers = ['aapl','adbe','amd','fi', 'csco', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'now']

optimal = optimal_sharpe(tickers, "2020-01-01", "2022-01-01", 0, 1000000)

print(optimal.fun)

[*********************100%***********************]  18 of 18 completed

1 Failed download:
['FI']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))


nan


/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/3127684956.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


In [125]:
def calculate_std(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change().dropna()

    # Calculate standard deviation of returns
    std = pd.DataFrame(returns.std(), columns=['Standard Deviation'])

    # Sort by standard deviation
    std_sorted = std.sort_values(by='Standard Deviation', ascending=False)

    # Add Rank column
    std_sorted['Rank'] = range(len(std_sorted))

    # Add Score column
    highest_std_value = std_sorted['Standard Deviation'].iloc[0]
    std_sorted['Score'] = (std_sorted['Standard Deviation'] / highest_std_value) * 100

    return std_sorted

In [126]:
# Loading data into variables
stock_filter = filter_stocks(ticker_lst)
ticker_data = stock_filter[0]
ticker_lst = list(ticker_data.keys()) # Reassign original ticker list
data = pd.DataFrame()
for ticker in ticker_data:
    data[ticker] = ticker_data[ticker]

# returns = data.pct_change()
# returns.drop(index=returns.index[0], inplace = True)

data.head()

$AGN: possibly delisted; no timezone found
$AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no timezone found
$CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no timezone found
$MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no timezone found
$RTN: possibly delisted; no price data found  (period=5d)


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-03,196.872959,177.616343,133.189175,357.399235,65.756941,162.104532,190.010096,176.331353,41.120219,6.53,...,156.439036,114.575356,37.799999,25.033068,77.623268,209.124092,700.023997,267.230890,207.384624,52.601339
2022-10-04,201.917482,182.329018,137.953580,370.108998,69.139715,169.392800,197.392274,186.767136,42.826401,6.74,...,163.312101,117.543648,42.610001,25.316833,79.079201,216.071333,710.425669,274.063121,214.759886,53.942628
2022-10-05,202.332054,184.049680,138.355068,370.149427,68.591525,169.196955,195.913115,184.808684,42.217995,6.67,...,166.675216,117.041313,41.959999,25.104010,78.620857,219.538422,715.721792,267.711270,213.539178,53.398525
2022-10-06,200.991492,180.146033,137.110432,363.578592,67.869502,168.287678,193.212650,184.934580,41.609586,6.64,...,167.195695,113.762512,41.320000,24.421211,75.751678,217.889886,704.939742,264.495333,210.576377,51.981321
2022-10-07,193.611385,178.181380,136.227155,350.410135,66.465586,160.257984,188.625946,181.563226,40.670527,6.29,...,161.363597,110.520241,37.349998,24.376873,74.671204,208.391429,685.548618,261.039173,202.361810,50.867795


In [127]:
# Function to get the total volume for a call or put of a given stock.
# ticker: yfinance Ticker class
# put: Boolean for if you want to calculate put volume. Else, put False for call volume. 
def get_options_vol(ticker, put):
    exps = ticker.options # Expiration dates of available options
    optdata = pd.DataFrame() # Data storage
    for exp in exps:
        chain = pd.DataFrame()
        if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
        else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
        optdata = pd.concat([optdata, chain]) # Add the calls/puts to the main dataframe. 
    return optdata.sum()['volume'] # output total volue of put/call options

# Function to calculate the PCR for each stock. 
def PCR_calc(tickers):
    pcrdata = pd.DataFrame(columns=['Ticker', 'Put Volume', 'Call Volume', 'PCR'])
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try: 
            # Get the volume for Put and Call options:
            call_options = get_options_vol(stock, False)
            put_options = get_options_vol(stock, True)
            # Calculate PCR Ratio:
            pcr = call_options / put_options # Order reversed from the formula for sake of ranking
            #print(f"Ticker: {ticker}, PCR: {pcr}")  # Debugging
            pcrdata.loc[len(pcrdata)] = [ticker, put_options, call_options, pcr]
        except Exception as e:
            print(f"Options Data Not Found {ticker}: {e} not found")  # Debugging (output error)
            pass
    return pcrdata

In [128]:
std = calculate_std(data)
std

/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/2355861676.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


,Standard Deviation,Rank,Score
SHOP.TO,0.036151,0,100.000000
BB.TO,0.034589,1,95.681598
PYPL,0.023819,2,65.888938
QCOM,0.023439,3,64.837266
AMZN,0.021313,4,58.955341
USB,0.021239,5,58.752268
BA,0.020323,6,56.217049
LLY,0.018084,7,50.025305
CAT,0.017586,8,48.645428
TXN,0.016667,9,46.104337


In [129]:
# Load the PCR values for each of the valid stocks into a variable
options_data = PCR_calc(ticker_lst)
options_data = options_data.sort_values(by='PCR', ascending=False)
options_data['Rank'] = [i for i in range(len(options_data))]
highest_pcr = options_data['PCR'].iloc[0]
options_data['Score'] = (options_data['PCR'] / highest_pcr) * 100
options_data.set_index('Ticker', inplace=True)


# Display the table of rankings based off PCR. 
# The rankings are based off the stocks with the greatest sentiment for if they will go up or not
# The tickers at the top of the list have a high call rate (meaning the price will go up)
pcr = options_data
pcr

Options Data Not Found BB.TO: 'volume' not found
Options Data Not Found RY.TO: 'volume' not found
Options Data Not Found SHOP.TO: 'volume' not found
Options Data Not Found T.TO: 'volume' not found
Options Data Not Found TD.TO: 'volume' not found


,Put Volume,Call Volume,PCR,Rank,Score
Ticker,,,,,
KO,11042.0,32018.0,2.899656,0,100.000000
PYPL,9597.0,26227.0,2.732833,1,94.246811
PEP,4818.0,12497.0,2.593815,2,89.452507
PG,5380.0,13729.0,2.551859,3,88.005572
AMZN,132738.0,264886.0,1.995555,4,68.820414
MRK,8463.0,14821.0,1.751270,5,60.395796
BA,25578.0,44716.0,1.748221,6,60.290642
PM,3807.0,6436.0,1.690570,7,58.302436
LLY,26829.0,43264.0,1.612583,8,55.612924


In [130]:
def calculate_scoreboard(std, pcr):
    """
    Merges two DataFrames (std and pcr) on their index (assumed to be ticker names),
    calculates the average of their 'Score' columns, and sorts the result by 'Average Score'.
    """

    merged = std[['Score']].merge(pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer')

    # interesting case here, we need to consider stocks with nan from PCR strategy
    merged['Score_std'].fillna(merged['Score_pcr'], inplace=True)
    merged['Score_pcr'].fillna(merged['Score_std'], inplace=True)
    
    # Calculate the average score
    merged['Average Score'] = merged[['Score_std', 'Score_pcr']].mean(axis=1)
    
    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)
    
    return merged_sorted

calculate_scoreboard(std, pcr)

/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/1719902720.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['Score_std'].fillna(merged['Score_pcr'], inplace=True)
/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_80529/1719902720.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are sett

,Score_std,Score_pcr,Average Score
SHOP.TO,100.000000,100.000000,100.000000
BB.TO,95.681598,95.681598,95.681598
PYPL,65.888938,94.246811,80.067874
AMZN,58.955341,68.820414,63.887877
KO,24.000752,100.000000,62.000376
PEP,28.192914,89.452507,58.822710
BA,56.217049,60.290642,58.253845
PG,26.499762,88.005572,57.252667
QCOM,64.837266,49.308236,57.072751
LLY,50.025305,55.612924,52.819114


In [131]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [132]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---